In [1]:
import random
import numpy as np
import os
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn
from sklearn.cluster import DBSCAN
import torch.nn.functional as F
from itertools import islice
import matplotlib.pyplot as plt
import collections
from chargenv5 import Env
from experience_generation_model import main
from feature_generation_model import simsiam
from collections import deque

torch.autograd.set_detect_anomaly(True)
class ReplayBuffer:
    def __init__(self, capacity, eps=1, min_samples=5):
        self.buffer = collections.deque(maxlen=capacity)
        self.clusters = {}
        self.eps = eps
        self.min_samples = min_samples

    def add(self, experience):
        # experience is expected to be a tuple (feature_vector, action, reward, next_feature_vector, done)
        self.buffer.append(experience)

    def cluster(self, features):
        # Assumes the first element of each experience in the buffer is the feature vector for clustering

        if len(features) == 0:
            return
        labels = DBSCAN(eps=self.eps, min_samples=self.min_samples).fit_predict(features.cpu())
        self.clusters = {}  # Reset clusters
        for i, label in enumerate(labels):
            if label not in self.clusters:
                self.clusters[label] = []
            self.clusters[label].append(self.buffer[i])

    def sample(self, batch_size):
        # Ensure there's at least one sample per cluster
        num_clusters = len(self.clusters)
        if num_clusters == 0:
            return []

        samples_per_cluster = max(1, batch_size // num_clusters)
        samples = []

        for cluster in self.clusters.values():
            if len(cluster) < samples_per_cluster:
                samples += cluster
            else:
                samples += random.sample(cluster, samples_per_cluster)

        # If we don't have enough samples due to rounding, sample from the entire buffer
        while len(samples) < batch_size:
            samples.append(random.choice(self.buffer))

        return samples[:batch_size]

    def size(self):
        return len(self.buffer)
# class ReplayBuffer:
#     def __init__(self, capacity):
#         self.buffer = collections.deque(maxlen=capacity)

#     def add(self, e):
#         self.buffer.append(e)

#     def sample(self, batch_size):
#         transitions = random.sample(self.buffer, batch_size)
#         return transitions

#     def size(self):
#         return len(self.buffer)

class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 256)
        self.fc3 = torch.nn.Linear(256, 64)
        self.fc4 = torch.nn.Linear(64, action_dim)
        self.action_bound = action_bound
        # init.normal_(self.fc1.weight, mean=0.5, std=0.01)
        # init.zeros_(self.fc1.bias)
        #
        # # 初始化第二个全连接层
        # init.normal_(self.fc2.weight, mean=0.5, std=0.01)
        # init.zeros_(self.fc2.bias)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x)) * self.action_bound
        return x

class QValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim + action_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 256)
        self.fc3 = torch.nn.Linear(256, 64)
        self.fc_out = torch.nn.Linear(64, 1)
        # init.normal_(self.fc1.weight, mean=0.0, std=0.01)
        # init.zeros_(self.fc1.bias)
        #
        # # 初始化第二个全连接层
        # init.normal_(self.fc2.weight, mean=0.0, std=0.01)
        # init.zeros_(self.fc2.bias)
        # init.normal_(self.fc_out.weight, mean=0.0, std=0.01)
        # init.zeros_(self.fc_out.bias)

    def forward(self, x, a):
        cat = torch.cat([x, a], dim=1) # 拼接状态和动作
        x = F.relu(self.fc1(cat))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc_out(x)

class DDPG:
    ''' DDPG算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device):
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.critic2 = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.lr = actor_lr
        
        self.actor.load_state_dict(torch.load("./model/actor_initial5.pth"))
        self.critic.load_state_dict(torch.load("./model/critic_initial5.pth" ))
        self.critic2.load_state_dict(torch.load("./model/critic2_initial5.pth" ))
        # init.normal_(self.actor.weight, mean=0.0, std=0.01)
        # init.zeros_(self.actor.bias)
        self.target_actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)
        self.target_critic = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        self.target_critic2 = QValueNet(state_dim, hidden_dim, action_dim).to(device)
        # self.actor.load_state_dict(torch.load("./model/train/actor_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # self.critic.load_state_dict(torch.load("./model/train/critic_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # self.critic2.load_state_dict(torch.load("./model/train/critic2_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # self.target_actor.load_state_dict(torch.load("./model/train/target_actor_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # self.target_critic.load_state_dict(torch.load("./model/train/target_critic_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # self.target_critic2.load_state_dict(torch.load("./model/train/target_critic2_DCE-DDPG%d_0.002.pth" % arrival_rate))
        # 初始化目标价值网络并设置和价值网络相同的参数

        self.target_actor.load_state_dict(self.actor.state_dict())
        self.target_critic.load_state_dict(self.critic.state_dict())
        self.target_critic2.load_state_dict(self.critic2.state_dict())
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=self.lr, weight_decay=0.001)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=self.lr, weight_decay=0.001)
        self.critic2_optimizer = torch.optim.Adam(self.critic2.parameters(), lr=self.lr, weight_decay=0.001)

        # 初始化第二个目标评论者网络的参数

        # 第二个评论者网络的优化器

        self.gamma = gamma
        self.sigma = sigma  # 高斯噪声的标准差,均值直接设为0
        self.tau = tau  # 目标网络软更新参数
        self.action_dim = action_dim
        self.device = device


    def decay_sigma(self):
        self.sigma *= 0.98
    def decay_lr(self):
        self.lr *= 0.5
    def take_action(self, state):
        state = state.to(self.device)
        action = self.actor(state).cpu()
        # 给动作添加噪声，增加探索
        action = action.detach() + self.sigma * torch.randn(self.action_dim, dtype=torch.float32)
        return action

    def soft_update(self, net, target_net):
        for param_target, param in zip(target_net.parameters(), net.parameters()):
            param_target.data.copy_(param_target.data * (1.0 - self.tau) + param.data * self.tau)

    def update(self, transition_dict):
        stacked_tensors = torch.cat((transition_dict), dim=1).squeeze(0).to(self.device)  # 移除中间的维度，得到 64x11
        states = stacked_tensors[:, :4]  # 结果形状为 64x4
        actions = stacked_tensors[:, 4:5]  # 结果形状为 64x1
        rewards = stacked_tensors[:, 5:6]  # 结果形状为 64x1
        next_states = stacked_tensors[:, 6:10]  # 结果形状为 64x4
        dones = stacked_tensors[:, 10:11]  # 结果形状为 64x1
        # next_q_values = self.target_critic(next_states, self.target_actor(next_states))
        # next_q_values = self.normalize(next_q_values)
        target_Q1 = self.target_critic(next_states, self.target_actor(next_states))
        target_Q2 = self.target_critic2(next_states, self.target_actor(next_states))
        target_Q = torch.min(target_Q1, target_Q2)
        q_targets = rewards + self.gamma * target_Q * (1 - dones)
        # q_targets = rewards + self.gamma * next_q_values * (1 - dones)
        critic_loss = F.mse_loss(self.critic(states, actions), q_targets, reduction='none')
        td_error1 = critic_loss.clone()
        critic_loss = (critic_loss).mean()
        self.critic_optimizer.zero_grad()
        critic_loss.backward(retain_graph=True)
        self.critic_optimizer.step()

        critic2_loss = F.mse_loss(self.critic2(states, actions), q_targets, reduction='none')
        td_error2 = critic2_loss.clone()
        critic2_loss = (critic2_loss).mean()
        self.critic2_optimizer.zero_grad()
        critic2_loss.backward()
        self.critic2_optimizer.step()

        # 在soft_update方法中也要更新第二个目标评论者网络
        if torch.equal(target_Q, target_Q1):
            actor_loss = -torch.mean(self.critic(states, self.actor(states)))
        else :
            actor_loss = -torch.mean(self.critic2(states, self.actor(states)))
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.soft_update(self.actor, self.target_actor)  # 软更新策略网络
        self.soft_update(self.critic, self.target_critic)  # 软更新价值网络
        self.soft_update(self.critic2, self.target_critic2)

        

def train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size, device):
    return_list = []
    max_reward = 0
    first = True

    for i_episode in range(num_episodes):
        episode_return = 0
        state = env.reset()
        # if i%10==0:
        agent.decay_sigma()
        # state = torch.tensor(state, dtype=torch.float32).to(device)
        done = False

        while not done:
            action = agent.take_action(state).to(device)
            next_state, reward, done = env.step(torch.transpose(action, 0, 1))
            # next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
            experience = torch.cat((state, action, reward.view(1,1), next_state,
                                    torch.tensor(done, dtype=torch.float32, device=device).view(1, 1)),dim=1)
            replay_buffer.add(experience.unsqueeze(1))

            state = next_state
            # print(env.price, reward)

            episode_return += reward.item()
            if replay_buffer.size() > minimal_size:
                if first:
                    if os.path.isfile('./model/cm_model%d.pth' % arrival_rate):
                        E = main.generate(arrival_rate)
                        E = torch.cat(E,dim=0).squeeze(1)

                        for i in range(E.size(0)):  # 遍历500
                            for j in range(E.size(1)):  # 遍历24
                                E[i, j, 3] = j+1
                                E[i, j, 9] = j+1
                                if j != E.size(1) - 1:
                                    E[i, j, -1] = 0  # 如果不是最后一个1x11的tensor，则最后一个值改为0
                                else:
                                    E[i, j, -1] = 1  # 如果是最后一个1x11的tensor，则最后一个值改为1
                                # 将修改后的 tensor 添加到经验池中
                                replay_buffer.add(E[i, j].unsqueeze(0).unsqueeze(0))
                                first = False
                        del E
                        # simsiam.train(replay_buffer.buffer, arrival_rate)
                        model = simsiam.test(arrival_rate)
                        features = model(torch.cat(list(replay_buffer.buffer), dim=0))[-1]
                        replay_buffer.cluster(features)
                        del model
                    else:
                        e = replay_buffer.sample(minimal_size)
                        num_tensors_to_concatenate = len(e) - (len(e) % 24)
                        data = [torch.cat(e[i:i + 24], dim=1) for i in range(0, num_tensors_to_concatenate, 24)]
                        main.train(data, arrival_rate)

                        E = main.generate(arrival_rate)
                        E = torch.cat(E, dim=0).squeeze(1)
                        for i in range(E.size(0)):  # 遍历500
                            for j in range(E.size(1)):  # 遍历24
                                if j != E.size(1) - 1:
                                    E[i, j, -1] = 0  # 如果不是最后一个1x11的tensor，则最后一个值改为0
                                else:
                                    E[i, j, -1] = 1  # 如果是最后一个1x11的tensor，则最后一个值改为1
                                # 将修改后的 tensor 添加到经验池中
                                replay_buffer.add(E[i, j].unsqueeze(0).unsqueeze(0))
                                first = False
                        simsiam.train(replay_buffer.buffer, arrival_rate)
                        model = simsiam.test(arrival_rate)
                        features = model(torch.cat(list(replay_buffer.buffer), dim=0))[-1]
                        replay_buffer.cluster(features)
                        del model
                else:
                    e = replay_buffer.sample(batch_size)
                    agent.update(e)
                    # agent.update(e)

                # transition_dict = {'states': b_s, 'actions': b_a, 'next_states': b_ns, 'rewards': b_r, 'dones': b_d, 'weight':w, 'index':i}
                # agent.update(transition_dict)
                # print(cl, al)
        return_list.append(episode_return)
        # if episode_return<150:
        #     torch.save(agent.actor.state_dict(), './model/actor_initial{}.pth'.format(arrival_rate))
        #     torch.save(agent.critic.state_dict(), './model/critic_initial{}.pth'.format(arrival_rate))
        #     torch.save(agent.critic2.state_dict(), './model/critic2_initial{}.pth'.format(arrival_rate))
        if max_reward < episode_return:
            # torch.save(agent.actor.state_dict(),'./model/train/actor_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            # torch.save(agent.critic.state_dict(), './model/train/critic_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            # torch.save(agent.critic2.state_dict(), './model/train/critic2_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            # torch.save(agent.target_actor.state_dict(), './model/train/target_actor_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            # torch.save(agent.target_critic.state_dict(), './model/train/target_critic_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            # torch.save(agent.target_critic2.state_dict(), './model/train/target_critic2_DCE-DDPG{}_{}.pth'.format(arrival_rate, actor_lr))
            max_reward = episode_return
        if (i_episode + 1) % 10 == 0:
            print('episodes: %d, reward: %f' % (i_episode + 1, torch.mean(torch.tensor(return_list[-10:],dtype=float))))
            if replay_buffer.size() > minimal_size:
                model = simsiam.test(arrival_rate)
                features = model(torch.cat(list(replay_buffer.buffer), dim=0))[-1]
                replay_buffer.cluster(features)
                del model
        if (i_episode + 1) % 500 == 0:
            agent.decay_lr()
    return return_list,max_reward


actor_lr = 0.002
critic_lr = 0.002
num_episodes = 1000

hidden_dim = 128
gamma = 0.98
tau = 0.001  # 软更新参数
buffer_size = 40000

minimal_size = 5000
batch_size = 128
sigma = 0.5  # 高斯噪声标准差
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


arrival_rate = 5
dataset = 'ACN-Data'
data_path = './datasets/{}.csv'.format(dataset)
env = Env(1, 0, 0, arrival_rate, data_path)

replay_buffer = ReplayBuffer(buffer_size)
state_dim = 4

action_dim = env.n_cs
action_bound = 0.5  # 动作最大值
#
agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device)

return_list, MR = train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size, device)
print(MR)

# episodes_list = list(range(len(return_list)))
# plt.plot(episodes_list, return_list)
# plt.xlabel('Episodes')
# plt.ylabel('Returns')
# plt.title('DCE-DDPG on DY')
# plt.show()

# with open('./result/DCE-DDPG{}_0.002.csv'.format(arrival_rate), 'w', encoding='utf-8') as file:
#     for item in return_list:
#         file.write(f"{item}\n")

episodes: 10, reward: 165.780208
episodes: 20, reward: 163.269290
episodes: 30, reward: 163.011146
episodes: 40, reward: 192.107413
episodes: 50, reward: 196.663525
episodes: 60, reward: 202.470552
episodes: 70, reward: 170.456361
episodes: 80, reward: 190.056116
episodes: 90, reward: 168.893784
episodes: 100, reward: 177.682407
episodes: 110, reward: 169.674088
episodes: 120, reward: 178.680290
episodes: 130, reward: 179.822204
episodes: 140, reward: 178.852478
episodes: 150, reward: 169.637723
episodes: 160, reward: 170.282537
episodes: 170, reward: 183.612440
episodes: 180, reward: 183.540196
episodes: 190, reward: 183.780551
episodes: 200, reward: 165.351042
episodes: 210, reward: 179.086543
episodes: 220, reward: 375.287060
episodes: 230, reward: 526.654875
episodes: 240, reward: 533.347036
episodes: 250, reward: 532.730057
episodes: 260, reward: 534.820509
episodes: 270, reward: 536.165038
episodes: 280, reward: 536.343437
episodes: 290, reward: 533.340484
episodes: 300, reward: 

In [30]:
import random
import numpy as np
import os
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch import nn
from sklearn.cluster import DBSCAN
import torch.nn.functional as F
from itertools import islice
import matplotlib.pyplot as plt
import collections
from chargenv import Env
import simsiam
from collections import deque
import time


class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 256)
        self.fc3 = torch.nn.Linear(256, 64)
        self.fc4 = torch.nn.Linear(64, action_dim)
        self.action_bound = action_bound
        # init.normal_(self.fc1.weight, mean=0.5, std=0.01)
        # init.zeros_(self.fc1.bias)
        #
        # # 初始化第二个全连接层
        # init.normal_(self.fc2.weight, mean=0.5, std=0.01)
        # init.zeros_(self.fc2.bias)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = torch.tanh(self.fc4(x)) * self.action_bound
        return x


class DDPG:
    ''' DDPG算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device):
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim, action_bound).to(device)    
        self.actor.load_state_dict(torch.load("./model/train/actor_DCE-DDPG{}_{}.pth".format(arrival_rate, actor_lr)))
        self.action_dim = action_dim
    def take_action(self, state):
        state = state.to(device)
        action = self.actor(state)
        # 给动作添加噪声，增加探索
        # action = action.detach() + self.sigma * torch.randn(self.action_dim, dtype=torch.float32)
        return action

        

def train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size, device):
    return_list = []
    max_reward = 0
    first = True

    for i_episode in range(num_episodes):
        episode_return = 0
        state = env.reset()

        # state = torch.tensor(state, dtype=torch.float32).to(device)
        done = False

        while not done:
            action = agent.take_action(state).to(device)
            next_state, reward, done = env.step(torch.transpose(action, 0, 1))
            
            state = next_state
            
            episode_return += reward.item()
            
        return_list.append(episode_return)
        print(episode_return)
    return return_list,max_reward



actor_lr = 0.002
critic_lr = 0.001
num_episodes = 100

hidden_dim = 128
gamma = 0.98
tau = 0.001  # 软更新参数
buffer_size = 10000

minimal_size = 300
batch_size = 128
sigma = 0.1 # 高斯噪声标准差
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

arrival_rate = 6
data_path = './datasets/EVCD{}.csv'.format(arrival_rate)
env = Env(1, 0, 0, arrival_rate, data_path)

replay_buffer = ReplayBuffer(buffer_size)
state_dim = 4

action_dim = env.n_cs
action_bound = 0.5  # 动作最大值
agent = DDPG(state_dim, hidden_dim, action_dim, action_bound, sigma, actor_lr, critic_lr, tau, gamma, device)



return_list, MR = train_off_policy_agent(env, agent, num_episodes, replay_buffer, minimal_size, batch_size, device)
end_time = time.time()
print('MR:',sum(return_list) / (len(return_list)),'Time:',(end_time-start_time)/num_episodes)



# os.makedirs(os.path.dirname('./result/test/DCE-DDPG{}.txt'.format(arrival_rate)), exist_ok=True)
#
# # 写入文件
# with open('./result/test/DCE-DDPG{}.txt'.format(arrival_rate), 'w', encoding='utf-8') as file:
#     file.write(f'Test_mode: DCE-DDPG{arrival_rate}, MR: {sum(return_list) / len(return_list)}, Time: {(end_time-start_time)/num_episodes}')


592.2731876373291
633.5940645933151
627.1296588182449
610.1912013292313
612.7230118513107
619.0970069169998
617.5239306688309
625.1366075277328
604.0409675836563
627.1296588182449
619.6823068857193
618.6710284948349
617.351046204567
623.9018400907516
617.4403344392776
616.6770249605179
613.803253531456
626.0424035787582
627.7026447057724
627.7049292325974
611.1629086732864
619.9348102807999
620.1604119539261
607.3999184370041
624.5854164361954
627.1827713251114
598.1810766458511
607.5437880754471
616.3633319139481
612.7159358263016
624.9088214635849
620.1604119539261
627.124459862709
621.4955102205276
608.0405181646347
630.4938875436783
621.2438470125198
613.8103266954422
619.9556204080582
617.351046204567
621.4903093576431
599.4374731779099
623.7321172952652
614.0456749200821
626.0495473146439
624.6098538637161
613.5537391901016
620.1604119539261
614.8847645521164
616.2689934968948
621.2438470125198
613.5499528646469
626.6979492902756
609.0554655790329
621.01675760746
616.612724661827